In [ ]:
import geopandas as gpd
import xLTS

class CrossingsLTSAgent(xLTS.Mixin):
    """
    Agent for calculating Level of Traffic Stress (LTS) for crossings using xLTS library.
    Inherits from xLTS.Mixin.
    """

    def run(self, input_file, output_file):
        """
        Reads the input geospatial file, processes it, and calculates LTS values.
        Saves the results to the specified output file.

        :param input_file: Path to the input file (e.g., SHP, GPKG, etc.)
        :param output_file: Path to save the output file (e.g., GeoJSON, GPKG, etc.)
        """
        # Read input file into a GeoDataFrame and fill missing values
        base_gdf = gpd.read_file(input_file).fillna(0)

        # Define columns to be converted to int and float types
        int_columns = [
            'ADT', 'Q85', 'NBLane', 'XDistSig', 'VisLim', 'RampTrig',
            'TwoStage', 'PocketTrig', 'FeuxVelo', 'SAS_Velo', 'AvancFlech'
        ]
        float_columns = ['VolRT', 'XileStop', 'c_SENS_CIR']

        # Convert columns to appropriate data types
        base_gdf[int_columns] = base_gdf[int_columns].astype(int)
        base_gdf[float_columns] = base_gdf[float_columns].astype(float)

        # Print the number of crossings
        print(f"{len(base_gdf)} CROSSINGS")

        # Apply LTS calculation to each row
        base_gdf['xlts'] = base_gdf.apply(
            lambda row: self.xLTS_calc(
                control=row['Control'],
                pocketTrig=row['PocketTrig'],
                xdistsig=row['XDistSig'],
                twostage=row['TwoStage'],
                linktype=row['LinkType'],
                speed_limit=row['Q85'],
                xllestop=row['XileStop'],
                adt=row['ADT'],
                nblane=row['NBLane'],
                vislim=row['VisLim'],
                c_sens_cir=row['c_SENS_CIR'],
                ramptrig=row['RampTrig'],
                feuxvelo=row['FeuxVelo'],
                sasvelo=row['SAS_Velo'],
                volrt=row['VolRT'],
                avancflech=row['AvancFlech']
            ), axis=1
        )

        # Rename specific columns
        base_gdf = base_gdf.rename(columns={
            'f_ONEWAY': 'f_DIVIDED',
            't_ONEWAY': 't_DIVIDED',
            'c_ONEWAY': 'c_DIVIDED'
        })

        # Save the results to the specified output file
        base_gdf.to_file(output_file, driver='GeoJSON')
        # If saving as GPKG, uncomment the following lines and adjust as needed
        # base_gdf.to_file(output_file, layer='xLTS_2', driver='GPKG')

        return base_gdf

# Example usage:
input_filepath = 'C:\\Users\\natdave\\folders\\Research\\Montreal\\cross_dec16.gpkg'
output_filepath = 'C:\\Users\\natdave\\folders\\Research\\Montreal\\xLTS_2.geojson'

agent = CrossingsLTSAgent()
lts_df = agent.run(input_filepath, output_filepath)
